### Analysis description
Launch a webgl visualizer and save brain images

In [1]:
# Imports
import os
import numpy as np
import nibabel as nb
import shutil
import warnings
import cortex
import time
import matplotlib.pyplot as plt
from pathlib import Path
from pycortex_utils import draw_cortex
warnings.filterwarnings('ignore')

# Define parameters
subjects = ['sub-001','sub-002','sub-003','sub-004',
            'sub-005','sub-006','sub-007','sub-008']
tasks = ['FullScreenAttendFix', 
         'FullScreenAttendBar',
         'FullScreen']

# Define folders
# base_dir = '/Users/martinszinte/disks/meso_S/data/gaze_prf'
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)
pycortex_dir = "{}/cortex".format(pp_dir)

# Plot parameters
cmap_polar, cmap_uni, cmap_ecc_size, cmap_2d = 'hsv', 'Reds', 'Spectral', 'PU_RdBu_covar_alpha'
save_svg = False
xfm_name = 'identity.fmriprep'
amp_th = 0
r2_th = [0.05, 1]
ecc_th = [0.1, 20]
sd_th = [0.1, 20]
alpha_range = [0, 0.4]
r2_range = [0, 1]
r2_2d_range = [0, 0.4]
ecc_range = [0, 10]
size_range = [0, 10]
col_offset = 1.0/14.0
cmap_steps = 255
recache = False

In [2]:
# Write pycortex config file
pycortex_db = "{}/db/".format(pycortex_dir)
pycortex_cm = "{}/colormaps/".format(pycortex_dir)
pycortex_config_file  = cortex.options.usercfg
pycortex_config_file_new = pycortex_config_file[:-4] + '_new.cfg'
pycortex_config_file_old = pycortex_config_file[:-4] + '_old.cfg'

Path(pycortex_config_file_new).touch()
with open(pycortex_config_file, 'r') as fileIn:
    with open(pycortex_config_file_new, 'w') as fileOut:
        for line in fileIn:
            if 'filestore' in line:
                newline = 'filestore=' + pycortex_db
                fileOut.write(newline)
                newline = '\n'
            elif 'colormaps' in line:
                newline = 'colormaps=' + pycortex_cm
                fileOut.write(newline)
                newline = '\n'
            else:
                newline = line
            fileOut.write(newline)
os.rename(pycortex_config_file,pycortex_config_file_old)
os.rename(pycortex_config_file_new, pycortex_config_file)

In [3]:
# create the local viewer 
# set parameters
# plot figure
# Create param matrix
# for subject in subjects:
for subject in ['sub-003']:
    maps_names = []
    volumes = {}
    for task in tasks:
        
        # Define and create folder
        fit_dir = '{}/{}/prf/fit'.format(pp_dir, subject)
        inflated_dir = '{}/{}/prf/inflated'.format(pp_dir, subject)
        try: os.makedirs(inflated_dir) 
        except: pass

        # Load fit parameters
        r2_im = nb.load('{}/{}_task-{}_par-r2.nii.gz'.format(fit_dir,subject,task))
        ecc_im = nb.load('{}/{}_task-{}_par-ecc.nii.gz'.format(fit_dir,subject,task))
        sd_im = nb.load('{}/{}_task-{}_par-sd.nii.gz'.format(fit_dir,subject,task))
        amp_im = nb.load('{}/{}_task-{}_par-amplitude.nii.gz'.format(fit_dir,subject,task))
        x_im = nb.load('{}/{}_task-{}_par-x.nii.gz'.format(fit_dir,subject,task))
        y_im = nb.load('{}/{}_task-{}_par-y.nii.gz'.format(fit_dir,subject,task))

        # Create param matrix
        r2_idx, ecc_idx, sd_idx, amp_idx, x_idx, y_idx = 0, 1, 2, 3, 4, 5
        param_mat = np.zeros((r2_im.shape[0],r2_im.shape[1],r2_im.shape[2],6))*np.nan
        param_mat[...,r2_idx] = r2_im.get_fdata()
        param_mat[...,ecc_idx] = ecc_im.get_fdata()
        param_mat[...,sd_idx] = sd_im.get_fdata()
        param_mat[...,amp_idx] = amp_im.get_fdata()
        param_mat[...,x_idx] = x_im.get_fdata()
        param_mat[...,y_idx] = y_im.get_fdata()

        # Threshold param matrix by masking R2
        param_mat_th = param_mat
        amp_down =  param_mat_th[...,amp_idx] > amp_th
        r2_th_down, r2_th_up = param_mat_th[...,r2_idx] >= r2_th[0],  param_mat_th[...,r2_idx] <= r2_th[1]
        ecc_th_down, ecc_th_up = param_mat_th[...,ecc_idx] >= ecc_th[0], param_mat_th[...,ecc_idx] <= ecc_th[1]
        sd_th_down, sd_th_up = param_mat_th[...,sd_idx] >= sd_th[0], param_mat_th[...,sd_idx] <= sd_th[1]
        all_th = np.array((amp_down, r2_th_down, r2_th_up, ecc_th_down, ecc_th_up, sd_th_down, sd_th_up)) 
        param_mat[np.logical_and.reduce(all_th)==False, r2_idx]=0
        
        # get R2 across tasks
        if task == 'FullScreenAttendFix': 
            r2_attend_mat = np.zeros((r2_im.shape[0],r2_im.shape[1],r2_im.shape[2],2))*np.nan
            r2_attend_mat[...,0] = param_mat[...,r2_idx]
        elif task == 'FullScreenAttendBar': 
            r2_attend_mat[...,1] = param_mat[...,r2_idx]
        
        # Define pRF R2
        r2_data = param_mat[...,r2_idx]
        alpha = (r2_data - alpha_range[0])/(alpha_range[1]-alpha_range[0])
        alpha[alpha>1]=1
        r2_param = {'data': param_mat[...,r2_idx], 'cmap': cmap_uni, 'alpha': alpha, 'vmin': r2_range[0],'vmax': r2_range[1],
                    'cbar': 'discrete', 'cbar_label': 'R2', 'description': '{}: pRF R2'.format(task), 'curv_brightness': 1, 
                    'curv_contrast': 0.5, 'add_roi': False, 'subject': subject, 'xfmname': xfm_name, 'save_fig':False}
        exec('{}_r2_param = r2_param'.format(task))
        maps_names.append('{}_r2'.format(task))
        
        # Define pRF eccentricity
        ecc_data = param_mat[...,ecc_idx]
        ecc_param = {'data': ecc_data, 'cmap': cmap_ecc_size, 'alpha': alpha, 'vmin': ecc_range[0], 'vmax': ecc_range[1],
                     'cbar': 'ecc','roi_name': 'pRF_eccentricity','description': '{}: pRF eccentricity'.format(task), 'curv_brightness': 1, 
                     'curv_contrast': 0.5, 'add_roi': save_svg, 'subject': subject, 'xfmname': xfm_name, 'save_fig':False}
        exec('{}_ecc_param = ecc_param'.format(task))
        maps_names.append('{}_ecc'.format(task))
        
        # Define pRF polar angle
        pol_comp_num = param_mat[...,x_idx] + 1j * param_mat[...,y_idx]
        polar_ang = np.angle(pol_comp_num)
        ang_norm = (polar_ang + np.pi) / (np.pi * 2.0)
        ang_norm = np.fmod(ang_norm + col_offset,1)
        polar_param = {'data': ang_norm, 'cmap': cmap_polar, 'alpha': alpha, 'vmin': 0, 'vmax': 1, 'cmap_steps': cmap_steps, 
                       'cbar': 'polar', 'roi_name': 'pRF_polar_angle', 'col_offset': col_offset, 'description': '{}: pRF polar angle'.format(task), 
                       'curv_brightness': 0.1, 'curv_contrast': 0.25, 'add_roi': save_svg, 'subject': subject, 'xfmname': xfm_name, 'save_fig':False}
        exec('{}_polar_param = polar_param'.format(task))
        maps_names.append('{}_polar'.format(task))
        
        # Define pRF size
        size_data = param_mat[...,sd_idx]
        size_param = {'data': size_data, 'cmap': cmap_ecc_size, 'alpha': alpha, 'vmin': size_range[0], 'vmax': size_range[1], 
                      'cbar': 'discrete', 'cbar_label': 'pRF size (dva)', 'description': '{}: pRF size'.format(task), 'curv_brightness': 1, 
                      'curv_contrast': 0.5, 'add_roi': False, 'subject': subject, 'xfmname': xfm_name, 'save_fig':False}
        exec('{}_size_param = size_param'.format(task))
        maps_names.append('{}_size'.format(task))
        
        # Define R2 attention comparison
        if task == 'FullScreen':
            attcmp_param = {'data':  r2_attend_mat[...,0], 'alpha': r2_attend_mat[...,1], 'cmap': cmap_2d, 'vmin': [r2_2d_range[0],r2_2d_range[0]],
                            'vmax': [r2_2d_range[1],r2_2d_range[1]], 'cortex_type': 'Volume2D', 'cbar': '2D', 'add_roi': False,
                            'description': 'Fullscreen: AttendFix vs. AttendBar', 'curv_brightness': 1, 'curv_contrast': 0.5,
                            'cbar_label': ['AttendFix','AttednBar'], 'subject': subject, 'xfmname': xfm_name, 'save_fig':False}
            exec('{}_attcmp_param = attcmp_param'.format(task))
            maps_names.append('{}_attcmp'.format(task))


In [16]:
# Flatmaps
maps2plot_names = ['FullScreen_polar','FullScreen_ecc','FullScreen_attcmp']

for maps_name in maps2plot_names:
    # Create flatmap
    exec('{}_volume, fig = draw_cortex(**{}_param)'.format(maps_name,maps_name))
    print("{}_task-{}".format(subject, maps_name))

    # Save flatmap in dataset
    exec('vol_description = {}_param["description"]'.format(maps_name))
    exec('volume = {}_volume'.format(maps_name))
    volumes.update({vol_description:volume})

sub-003_task-FullScreen_polar
sub-003_task-FullScreen_ecc
sub-003_task-FullScreen_attcmp


In [17]:
port_num = 25971
print("To type locally...")
print("ssh -Y -fN -L localhost:{}:localhost:{} admin@invibe.nohost.me".format(port_num,port_num))
print("Go to: http://localhost:{}/".format(port_num))
handle = cortex.webgl.show(data=volumes,recache = False, port=port_num)
# to set manually
# take out roi


To type locally...
ssh -Y -fN -L localhost:25971:localhost:25971 admin@invibe.nohost.me
Go to: http://localhost:25971/
Started server on port 25971


In [15]:
# Manual edit
# 1/ Pick polar angle dataset
# 2/ Take out labels
# 3/ Run cell

inflated_polar = {  'camera.azimuth':220,
                    'camera.altitude':73,
                    'camera.radius':324,
                    'surface.{subject}.unfold':0.45,
                    'surface.{subject}.pivot':34,
                    'surface.{subject}.left':True,
                    'surface.{subject}.depth':0.5,
                    'surface.{subject}.right':True,
                    'surface.{subject}.specularity':0.2,
                    'surface.{subject}.dither':False,
                    'surface.{subject}.colorbar':False,
                    'surface.{subject}.sampler':'trilinear',
                    'surface.{subject}.curvature.brightness':0.1,
                    'surface.{subject}.curvature.contrast':0.25,
                    'surface.{subject}.curvature.smoothness':0.1,
                    }
handle._set_view(**inflated_polar)
time.sleep(5)
handle.getImage('{}/{}_inflated_polar.png'.format(inflated_dir,subject),size = (3000, 3000))



[{}]

Stopping server


In [20]:
# Manual edit
# 1/ Pick eccentricty dataset
inflated_ecc = {'camera.azimuth':220,
                'camera.altitude':73,
                'camera.radius':324,
                'surface.{subject}.unfold':0.45,
                'surface.{subject}.pivot':34,
                'surface.{subject}.left':True,
                'surface.{subject}.depth':0.5,
                'surface.{subject}.right':True,
                'surface.{subject}.specularity':0.2,
                'surface.{subject}.dither':False,
                'surface.{subject}.colorbar':False,
                'surface.{subject}.sampler':'trilinear',
                'surface.{subject}.curvature.brightness':1,
                'surface.{subject}.curvature.contrast':0.5,
                'surface.{subject}.curvature.smoothness':0.1,
                }
handle._set_view(**inflated_ecc)
time.sleep(5)
handle.getImage('{}/{}_inflated_ecc.png'.format(inflated_dir,subject),size = (3000, 3000))


[{}]

In [21]:
# Manual edit
# 1/ Pick R2 comparision dataset
inflated_attcmp = { 'camera.azimuth':220,
                    'camera.altitude':73,
                    'camera.radius':324,
                    'surface.{subject}.unfold':0.45,
                    'surface.{subject}.pivot':34,
                    'surface.{subject}.left':True,
                    'surface.{subject}.depth':0.5,
                    'surface.{subject}.right':True,
                    'surface.{subject}.specularity':0.2,
                    'surface.{subject}.dither':False,
                    'surface.{subject}.colorbar':False,
                    'surface.{subject}.sampler':'trilinear',
                    'surface.{subject}.curvature.brightness':1,
                    'surface.{subject}.curvature.contrast':0.5,
                    'surface.{subject}.curvature.smoothness':0.1,
                    }
handle._set_view(**inflated_attcmp)
time.sleep(5)
handle.getImage('{}/{}_inflated_attcmp.png'.format(inflated_dir,subject),size = (3000, 3000))


[{}]

Stopping server
